# Sample satellite data

Some graphical tests of current primary functions using the sample satellite data.

In [ ]:
import sys; sys.path.append("../")
import warnings

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import tams

In [ ]:
plt.close("all")

xr.set_options(display_expand_data=False)

## Load data

In [ ]:
r = tams.load_example_ir()

tb = tams.tb_from_ir(r, ch=9).isel(time=slice(4))
tb

In [ ]:
tb.isel(time=0).plot(x="lon", y="lat", size=5, aspect=2.5)

## Identify cloud elements (CEs)

In [ ]:
times = tb.time
contour_sets, contour_sets_219 = tams.identify(tb)
contour_sets[0].head()

In [ ]:
contour_sets_219[0].head()

In [ ]:
# Simple plot to test 219 matching
m, n = 0, 1  # time, contour #
fig, ax = plt.subplots()
c = contour_sets[m].iloc[[n]]
c.plot(ax=ax)
c.cs219.plot(color="red", ax=ax, alpha=0.4)

## Track CE groups between times

In [ ]:
cs = tams.track(contour_sets, times, u_projection=-5).reset_index(drop=True)
cs.head()

In [ ]:
tams.plot_tracked(cs)

## Classify

In [ ]:
cs = tams.classify(cs)
cs.head()